In [39]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read data
student_data = pd.read_csv("student-mat.csv")

In [25]:
# student_data.head()
# student_data.notnull()
# student_data.columns

In [38]:
# Number of students
# n_students = student_data.shape[0]
# print ("Total number of students: {}".format(n_students))

In [61]:
# Extract feature columns
feature_cols = list(student_data.columns[:-3])
#feature_cols

# Extract target column 'passed'
#target_col = student_data.columns[-3:] 
target_col_G1 = student_data.columns[-3]
target_col = target_col_G1
#target_col

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

In [62]:
y_all

0       5
1       5
2       7
3      15
4       6
       ..
390     9
391    14
392    10
393    11
394     8
Name: G1, Length: 395, dtype: int64

In [41]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables 
    into binary (0/1) variables. Converts categorical variables into dummy 
    variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ("Processed feature columns ({} total features):"\
      "\n\n{}".format(len(X_all.columns), ",   ".join(X_all.columns)))

Processed feature columns (48 total features):

school_GP,   school_MS,   sex_F,   sex_M,   age,   address_R,   address_U,   famsize_GT3,   famsize_LE3,   Pstatus_A,   Pstatus_T,   Medu,   Fedu,   Mjob_at_home,   Mjob_health,   Mjob_other,   Mjob_services,   Mjob_teacher,   Fjob_at_home,   Fjob_health,   Fjob_other,   Fjob_services,   Fjob_teacher,   reason_course,   reason_home,   reason_other,   reason_reputation,   guardian_father,   guardian_mother,   guardian_other,   traveltime,   studytime,   failures,   schoolsup,   famsup,   paid,   activities,   nursery,   higher,   internet,   romantic,   famrel,   freetime,   goout,   Dalc,   Walc,   health,   absences


In [42]:
X_all

,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,1,0,1,0,18,0,1,1,0,1,...,1,0,0,4,3,4,1,1,3,6
1,1,0,1,0,17,0,1,1,0,0,...,1,1,0,5,3,3,1,1,3,4
2,1,0,1,0,15,0,1,0,1,0,...,1,1,0,4,3,2,2,3,3,10
3,1,0,1,0,15,0,1,1,0,0,...,1,1,1,3,2,2,1,1,5,2
4,1,0,1,0,16,0,1,1,0,0,...,1,0,0,4,3,2,1,2,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,0,1,0,1,20,0,1,0,1,1,...,1,0,0,5,5,4,4,5,4,11
391,0,1,0,1,17,0,1,0,1,0,...,1,1,0,2,4,5,3,4,2,3
392,0,1,0,1,21,1,0,1,0,0,...,1,0,0,5,5,3,3,3,3,3
393,0,1,0,1,18,1,0,0,1,0,...,1,1,0,4,4,1,3,4,5,0


In [58]:
# import bnlearn as bn
# # Learn the DAG in data using Bayesian structure learning:
# DAG = bn.structure_learning.fit(X_all)

# # print adjacency matrix
# print(DAG['adjmat'])
# # target     Cloudy  Sprinkler   Rain  Wet_Grass
# # source                                        
# # Cloudy      False      False   True      False
# # Sprinkler    True      False  False       True
# # Rain        False      False  False       True
# # Wet_Grass   False      False  False      False

# # Plot
# G = bn.plot(DAG)

# # Interactive plotting
# G = bn.plot(DAG, interactive=True)

In [74]:
import os
import random
import warnings
import numpy as np
import pandas as pd
import networkx as nx
import statsmodels
from scipy.io import arff
from scipy.signal._signaltools import _centered
import matplotlib.pyplot as plt
#import pgmpy.estimators as ests
from pgmpy.models import BayesianNetwork
#from pgmpy.metrics import structure_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# warnings.simplefilter("ignore")

In [75]:
import pgmpy.estimators as ests

ModuleNotFoundError: No module named 'pgmpy.estimators'

In [53]:
# import  scipy.signal.signaltools

# def _centered(arr, newsize):
#     # Return the center newsize portion of the array.
#     newsize = np.asarray(newsize)
#     currsize = np.array(arr.shape)
#     startind = (currsize - newsize) // 2
#     endind = startind + newsize
#     myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
#     return arr[tuple(myslice)]

# scipy.signal.signaltools._centered = _centered

In [63]:
est = ests.TreeSearch(pd.DataFrame(y_all), root_node=0)
model = est.estimate(estimator_type='chow-liu')
nx.draw_circular(model, 
                 with_labels=True, 
                 arrowsize=20, 
#                  arrowstyle='fancy', 
#                  alpha=0.3
                )

NameError: name 'ests' is not defined